# Lecture 5--Optimization

## 1. Data science: Logistic regression

### 1.1. Derivation

#### Linear formulation

$$\mathcal L=\prod_{i=1}^n p_i^{y_i}(1-p_i)^{1-y_i}$$
$$\mathcal L=\prod_{i=1}^n F(x_i'\beta)^{y_i}(1-F(x_i'\beta))^{1-y_i}$$

$$\ln\mathcal L=\sum_{i=1}^n y_i \ln{F(x_i'\beta)}+(1-y_i)\ln{(1-F(x_i'\beta))}$$
$$\ln\mathcal L=\left[\ln{F(\beta'\mathbf X')}\right]y+\left[\ln{(\mathbf{1}'-F(\beta'\mathbf X'))}\right](1-y)$$

In [12]:
expit = lambda x: 1/(1+np.exp(-x))
def loglike(x,y,b):
    Fx = expit(b.T@x.T)
    return np.log(Fx)@y+np.log(1-Fx)@(1-y)

#### Gradient
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\text{diag}\left(\frac{f(\mathbf X\beta)}{F(\mathbf X\beta)}\right)y-\mathbf X'\text{diag}\left(\frac{f(\mathbf X\beta)}{\mathbf 1-F(\mathbf X\beta)}\right)(1-y)$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\text{diag}\left(\frac{f(\mathbf X\beta)(1-F(\mathbf X\beta))}{(1-F(\mathbf X\beta))F(\mathbf X\beta)}\right)y-\mathbf X'\text{diag}\left(\frac{f(\mathbf X\beta)F(\mathbf X\beta)}{\mathbf F(\mathbf X\beta)(1-F(\mathbf X\beta))}\right)(1-y)$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\left[\text{diag}\left(1-F(\mathbf X\beta)\right)y-\text{diag}\left(F(\mathbf X\beta)\right)(1-y)\right]$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\left[\text{diag}\left(y-F(\mathbf X\beta)y-F(\mathbf X\beta)+F(\mathbf X\beta)y)\right)\right]\mathbf 1$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\left[\text{diag}\left(y-F(\mathbf X\beta)\right)\right]\mathbf 1$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\left[y-F(\mathbf X\beta)\right]$$

In [7]:
def gradient(x,y,b):
    Fx = expit(x@b)
    return x.T@(y-Fx)

#### Hessian
$$\frac{d}{d\beta}\frac{d\ln\mathcal L}{d\beta}'=\frac{d}{d\beta}\left[y'-F(\beta'\mathbf X')\right]\mathbf X$$
$$\frac{d^2\ln\mathcal L}{d\beta d\beta'}=-\mathbf X'\left[\text{diag}\left(f(\mathbf X\beta)\right)\right]\mathbf X$$

In [2]:
def hessian(x,y,b):
    Fx = expit(x@b)
    fx = Fx*(1-Fx)
    return -x.T@np.diagflat(fx.flatten())@x

**Theorem** Crammer-Rao Lower Bound

Assume
$\mathcal{L}$ is continuous and differentiable. For any unbiased estimator $\hat\theta$, the variance is bounded below by
$$\text{Var}\left[\hat\theta\right]\ge\left[-\text{E}\left[\frac{d^2\ln{\mathcal{L}}}{d\theta d\theta'}\right]\right]^{-1}$$


## 2. Programming

### 2.1 Grid search

Search over a given parameter space. Check every possible option for the optimum value

In [5]:
import numpy as np
from itertools import product

def grid_search(func,space,maximize=False):
    vstates = [(x,func(x)) for x in space]
    vstates.sort(key=lambda x: x[1])
    return vstates[-1][0] if maximize else vstates[0][0]

x = np.linspace(0,10,1000).tolist()
func = lambda x: (x[0]-4.0001)**2*(x[1]-6.0001)**2
grid_search(func,product(x,x))

(4.004004004004004, 5.995995995995996)

### 2.2 Gradient descent

Walk along the slope of the curve by steps proportional to the opposite of the size of the gradient. 

In [122]:
def gradient_descent(func,gradient,init_x:np.ndarray,learning_rate:float=0.05,max_reps:int=1000,maximize=False):
    x = init_x.copy()
    for i in range(max_reps):
        gx = gradient(x)
        x0 = x.copy()
        flast = func(x)
        x += gx*learning_rate if maximize else -gx*learning_rate
        if (func(x)<flast and maximize and i>2) or (func(x)>flast and (not maximize) and i>2): 
            x = x0
            break
    return x


def gradient_descent(func,gradient,init_x:np.ndarray,learning_rate:float=0.05,max_reps:int=1000,maximize=False):
    x = init_x.copy()
    for i in range(max_reps):
        x += gradient(x)*learning_rate if maximize else -gradient(x)*learning_rate
    return x

### 2.3 Newton's method

In [10]:
def newton(gradient,hessian,init_x:np.ndarray,max_reps:int=100,tolerance:float=1e-6):
    x = init_x.copy()
    for i in range(max_reps):
        update = -np.linalg.solve(hessian(x),gradient(x))
        x += update
        if np.abs(update).sum()<tolerance:
            return (x,i)
    raise Exception('Newton did not converge')

### 2.4 Complete code

In [7]:
from cleands import *

class likelihood_model(learning_model):
    def evaluate_lnL(self,pred): raise NotImplementedError
    @property
    def lnL(self): return self.evaluate_lnL(self.fitted)
    @property
    def aic(self): return 2*self.n_feat-2*self.lnL
    @property
    def bic(self): return np.log(self.n_obs)*self.n_feat-2*self.lnL
    @property
    def deviance(self): return 2*self.lnL-2*self._null_lnL_()
    def _gradient_(self,coef): raise NotImplementedError
    def _hessian_(self,coef): raise NotImplementedError
    def _null_lnL_(self): return self.evaluate_lnL(np.ones(self.y.shape)*self.y.mean())
    def __vcov_params_lnL__(self): return -np.linalg.inv(self._hessian_(self.params))
    def __max_likelihood__(self,init_params,gradient=None,hessian=None):
        if gradient==None: gradient=self._gradient_
        if hessian==None: hessian=self._hessian_
        return newton(gradient,hessian,init_params)
class linear_model(prediction_model,likelihood_model):
    def __init__(self,x,y):
        super(linear_model,self).__init__(x,y)
        self.params = self.__fit__(x,y)
    def __fit__(self,x,y): return np.linalg.solve(x.T@x,x.T@y)
    def predict(self,target): return target@self.params
    def evaluate_lnL(self,pred): return -self.n_obs/2*(np.log(2*np.pi*(self.y-pred).var())+1)
    @property
    def r_squared(self):
        return 1-self.residuals.var()/self.y.var()
    @property
    def adjusted_r_squared(self):
        return 1-(1-self.r_squared)*(self.n_obs-1)/(self.n_obs-self.n_feat)
    @property
    def degrees_of_freedom(self):
        return self.n_obs-self.n_feat
    @property
    def ssq(self):
        return self.residuals.var()*(self.n_obs-1)/self.degrees_of_freedom
class logistic_regressor(linear_model):
    def __fit__(self,x,y):
        params,self.iters = self.__max_likelihood__(np.zeros(self.n_feat))
        return params
    @property
    def vcov_params(self):return self.__vcov_params_lnL__()
    def evaluate_lnL(self,pred):return self.y.T@np.log(pred)+(1-self.y).T@np.log(1-pred)
    def _gradient_(self,coefs):return self.x.T@(self.y-expit(self.x@coefs))
    def _hessian_(self,coefs):
        Fx = expit(self.x@coefs)
        return -self.x.T@np.diagflat((Fx*(1-Fx)).values)@self.x
    def predict(self,target):return expit(target@self.params)
class LogisticRegressor(logistic_regressor,broom_model):
    def __init__(self,x_vars:list,y_var:str,data:pd.DataFrame,*args,**kwargs):
        super(LogisticRegressor,self).__init__(data[x_vars],data[y_var],*args,**kwargs)
        self.x_vars = x_vars
        self.y_var = y_var
        self.data = data
    def _glance_dict_(self):
        return {'mcfadden.r.squared':self.r_squared,
                'adjusted.r.squared':self.adjusted_r_squared,
                'self.df':self.n_feat,
                'resid.df':self.degrees_of_freedom,
                'aic':self.aic,
                'bic':self.bic,
                'log.likelihood':self.lnL,
                'deviance':self.deviance,
                'resid.var':self.ssq}

In [13]:
## Data generation
df = pd.DataFrame(np.random.normal(size=(500,4)),columns=['x1','x2','x3','y'])
df['y'] += df[['x1','x2','x3']]@np.random.uniform(size=(3,))
df['y'] = (df['y']>0).astype(int)

## Run the model
model = LogisticRegressor(*add_intercept(['x1','x2','x3'],'y',df))

## See table
model.tidy

,variable,estimate,std.error,t.statistic,p.value
0,(intercept),-0.09402624346059973,0.1039609294588749,-0.9044382726281304,0.36620212945570596
1,x1,0.3554820793267563,0.10452686459940479,3.4008680992118894,0.0007259445359066901
2,x2,1.2785220966621271,0.13418796797136462,9.527844530255951,7.115336778797292e-20
3,x3,0.17181062666232,0.10442145535365713,1.6453575185332132,0.10052962564100756


In [14]:
model.glance

,mcfadden.r.squared,adjusted.r.squared,self.df,resid.df,aic,bic,log.likelihood,deviance,resid.var
,0.250654,0.246121,4,496,556.809244,573.667676,-274.404622,142.536855,0.188168


## 3. Programming challenges

### 3.1 Recursive partitioning trees

Write a class that implements a recursive partitioning algorithm. Use our common machine learning code.

In [1]:
from cleands import *

class tree_based_model(prediction_model,likelihood_model):
    def __init__(self,x,y,level:int=0,max_level:int=1):
        super(tree_based_model,self).__init__(x,y)
        if level<max_level:
            RSS = [self.__calc_RSS_and_split__(x[:,i]) for i in range(self.n_feat)]
            RSS = np.array(RSS)
            self.split_variable = RSS[:,0].argmin()
            self.RSS = RSS[self.split_variable,0]
            self.split_value = RSS[self.split_variable,1]
            
            lowerx = x[x[:,self.split_variable]<self.split_value,:]
            lowery = y[x[:,self.split_variable]<self.split_value]
            upperx = x[x[:,self.split_variable]>=self.split_value,:]
            uppery = y[x[:,self.split_variable]>=self.split_value]
            self.lower = tree_based_model(lowerx,lowery,level+1,max_level)
            self.upper = tree_based_model(upperx,uppery,level+1,max_level)
        else:
            self.split_variable = None
            self.RSS = ((y-y.mean())**2).sum()
            self.split_value = None
    def __calc_RSS_and_split__(self,x):
        y = self.y
        outmat = []
        for item in np.unique(x):
            lowery = y[x<item].astype(float)
            uppery = y[x>=item].astype(float)
            if lowery.shape[0]<2 or uppery.shape[0]<2: continue
            lowery -= lowery.mean()
            uppery -= uppery.mean()
            RSS = (lowery**2).sum()+(uppery**2).sum()
            outmat += [(RSS,item)]
        outmat = np.array(outmat)
        return outmat[outmat[:,0].argmin(0)]
    def predict(self,target): 
        if self.split_variable == None:
            return np.full(shape=target.shape[0],fill_value=self.y.mean())
        outmat = np.zeros(target.shape[0])
        lowerx = target[target[:,self.split_variable]<self.split_value]
        upperx = target[target[:,self.split_variable]>=self.split_value]
        outmat[target[:,self.split_variable]<self.split_value] = self.lower.predict(lowerx)
        outmat[target[:,self.split_variable]>=self.split_value] = self.upper.predict(upperx)
        return outmat
    def evaluate_lnL(self,pred): return -self.n_obs/2*(np.log(2*np.pi*(self.y-pred).var())+1)

## Data generation
df = pd.DataFrame(np.random.normal(size=(500,4)),columns=['x1','x2','x3','y'])
df['y'] += df[['x1','x2','x3']]@np.random.uniform(size=(3,))*2
df['y'] = (df['y']>0).astype(int)

## Run the model
model = tree_based_model(df[['x1','x2','x3']].values,df['y'].values,max_level=4)

## See table
((model.fitted>0.5).astype(int)==df['y'].values).mean()

IndexError: too many indices for array

In [102]:
df = pd.DataFrame(np.random.normal(size=(500,4)),columns=['x1','x2','x3','y'])
df['y'] += df[['x1','x2','x3']]@np.random.uniform(size=(3,))
df['y'] = (df['y']>0).astype(int)

x = df[['x1','x2','x3']].values
y = df['y'].values

x,y

(array([[-0.9832525 , -0.69196483,  1.51658726],
        [ 1.53312844,  0.20947803, -0.68203532],
        [ 0.78611393, -0.57058064, -0.19872358],
        ...,
        [ 0.03813705, -1.13891395,  2.1349832 ],
        [ 0.26428675, -0.9581884 , -0.50058862],
        [ 0.50694873, -1.78798524, -0.03520808]]),
 array([1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 0,

In [156]:

#outmat

0.0 124.16064257028111 124.16064257028111
0.0 123.93561368209254 123.93561368209254
0.0 123.70967741935485 123.70967741935485
0.0 123.4828282828283 123.4828282828283
0.0 123.25506072874492 123.25506072874492
0.0 123.026369168357 123.026369168357
0.0 122.79674796747963 122.79674796747963
0.0 122.56619144602851 122.56619144602851
0.0 122.33469387755099 122.33469387755099
0.0 122.10224948875256 122.10224948875256
0.0 121.86885245901638 121.86885245901638
0.0 121.63449691991784 121.63449691991784
0.0 121.39917695473254 121.39917695473254
0.0 121.16288659793814 121.16288659793814
0.0 120.92561983471076 120.92561983471076
0.0 120.68737060041408 120.68737060041408
0.0 120.448132780083 120.448132780083
0.0 120.2079002079002 120.2079002079002
0.0 119.96666666666663 119.96666666666663
0.0 119.7244258872651 119.7244258872651
0.0 119.48117154811712 119.48117154811712
0.0 119.23689727463318 119.23689727463318
0.0 118.99159663865547 118.99159663865547
0.0 118.74526315789471 118.74526315789471
0.0 11

array([100.11829158,   0.31274329])

### 3.2 Quaternions

The Quaternions are a generalization of complex numbers. Where the complex numbers have two components, $a$ and $b$, for a number $a+bi$, the Quaternions have four parts $a, b, c$ and $d$: $$a+bi+cj+dk$$

The Quaternions have four basic operations: addition, subtraction, multiplication. and the inverse. Your job is to write a quaternion class which implements these operations. You can learn how to perform these operations on the Quaternions' wikipedia page.

## Problem set solutions

In [20]:
def hailstone(x,a,b):
    outlist = []
    for i in range(1000):
        outlist += [x]
        if x in outlist[:-1]: return outlist
        x = x//2 if x%2==0 else a*x+b
    return False

def converges(a,b):
    for x in range(1,1000):
        if hailstone(x,a,b):
            pass
        else:
            print('a={0}, b={1} does not converge'.format(a,b))
            return
    print('a={0}, b={1} converges'.format(a,b))
def cycles(a,b):
    cycle_list = []
    for x in range(1,1000):
        hs = hailstone(x,a,b)
        if hs:
            hs = min(hs[hs.index(hs[-1]):-1])
            if hs not in cycle_list: cycle_list += [hs]
        else: return
    print('a={0}, b={1} {2}'.format(a,b,cycle_list))

for a,b in product(range(1,11),range(1,11)):
    cycles(a,b)

a=1, b=1 [1]
a=1, b=3 [1, 3]
a=1, b=5 [1, 5]
a=1, b=7 [1, 3, 7]
a=1, b=9 [1, 3, 9]
a=2, b=2 [1]
a=2, b=6 [1, 3]
a=2, b=10 [1, 5]
a=3, b=1 [1]
a=3, b=3 [3]
a=3, b=5 [1, 19, 5, 23, 187, 347]
a=3, b=7 [5, 7]
a=3, b=9 [9]
a=4, b=4 [1]
a=6, b=2 [1]
a=6, b=6 [3]
a=6, b=10 [1, 19, 5, 23, 187, 347]
a=8, b=8 [1]


In [72]:
n = 1000
nsim = 1000
reject = 0
for isim in range(nsim):
    x = np.random.normal(size=(n,))
    e = np.random.normal(size=(n,))
    for t in range(1,n):
        x[t] += 0.75*x[t-1]
        e[t] += 0.75*e[t-1]
    y = e
    df = pd.DataFrame({'y':y,'x':x})
    model = LeastSquaresRegressor(['x'],'y',df)
    if np.abs(model.t.item())>1.96: reject += 1
print(reject/nsim)

0.288
